In [1]:
import pandas as pd
import pickle
import pymysql
import csv
import os 

In [2]:
data_path = './data'

In [3]:
with open(os.path.join(data_path, 'bws_set.pickle'), 'rb') as f:
    bws_set = pickle.load(f)

In [4]:
bws_dep = pd.read_csv(os.path.join(data_path, 'bws_dep_translated.csv'))
bws_ndep = pd.read_csv(os.path.join(data_path, 'bws_daily_translated.csv'))
bws_ndep.index += 1200 
bws = pd.concat([bws_dep, bws_ndep])
bws

,text,translated
0,male aged i used to feel depressed since the ...,남자 나이 나는 '비정상적인' 의미에서 나이 이후로 우울함을 느끼곤 했다
1,i am not really sure why i am depressed i am n...,나는 내가 왜 우울한지 잘 모르겠다. 나는 결코 인기가 없지만 나는 친구들이 있다
2,i don't know if i can really give you any soli...,나 자신이 우울하기 때문에 내가 정말로 너에게 확실한 조언을 해줄 수 있을지 모르겠다
3,which is the same when i am depressed but at l...,그것은 내가 우울할 때도 마찬가지이지만 적어도 의학적 적응 전에 나는 잠시 동안 진...
4,i don't think i am depressed i just suck,나는 내가 우울하다고 생각하지 않아. 그냥 형편없어
...,...,...
1595,Or lie down on the electric pad,아니면 전기장판 위에 눕는다
1596,Let's save money and go there,돈 모아서 거기 가자
1597,"But if you need to go with me, you can decide....",하지만 나랑 같이 갈 일이 있으면 결정해도 돼...
1598,It was really delicious.,꿀맛이었지요.


In [5]:
bws['idx'] = bws.index
bws['cnt'] = 0
bws['weakest_cnt'] = 0
bws['strongest_cnt'] = 0

In [9]:
bws = bws[['idx', 'text', 'translated', 'cnt', 'weakest_cnt', 'strongest_cnt']]
bws

,idx,text,translated,cnt,weakest_cnt,strongest_cnt
0,0,male aged i used to feel depressed since the ...,남자 나이 나는 '비정상적인' 의미에서 나이 이후로 우울함을 느끼곤 했다,0,0,0
1,1,i am not really sure why i am depressed i am n...,나는 내가 왜 우울한지 잘 모르겠다. 나는 결코 인기가 없지만 나는 친구들이 있다,0,0,0
2,2,i don't know if i can really give you any soli...,나 자신이 우울하기 때문에 내가 정말로 너에게 확실한 조언을 해줄 수 있을지 모르겠다,0,0,0
3,3,which is the same when i am depressed but at l...,그것은 내가 우울할 때도 마찬가지이지만 적어도 의학적 적응 전에 나는 잠시 동안 진...,0,0,0
4,4,i don't think i am depressed i just suck,나는 내가 우울하다고 생각하지 않아. 그냥 형편없어,0,0,0
...,...,...,...,...,...,...
1595,1595,Or lie down on the electric pad,아니면 전기장판 위에 눕는다,0,0,0
1596,1596,Let's save money and go there,돈 모아서 거기 가자,0,0,0
1597,1597,"But if you need to go with me, you can decide....",하지만 나랑 같이 갈 일이 있으면 결정해도 돼...,0,0,0
1598,1598,It was really delicious.,꿀맛이었지요.,0,0,0


In [10]:
len(bws.idx.unique())

1600

In [11]:
bws.to_csv(os.path.join(data_path, 'bws_db.csv'), index=False)

In [6]:
conn = pymysql.connect(host='localhost', user='cori', password='Dhwoehd12!', db='bws', charset='utf8')
curs = conn.cursor()

In [7]:
def execute_sql(sql):
    curs.execute(sql)
    return curs.fetchall()

In [8]:
sql = 'SHOW TABLES;'

execute_sql(sql)

(('bws_set',), ('bws_table',))

In [15]:
sql = "DROP TABLE bws_table"

execute_sql(sql)

()

In [9]:
sql = "CREATE TABLE bws_test(idx int NOT NULL, \
eng_text varchar(1000) NOT NULL, \
kor_text varchar(1000) NOT NULL, \
cnt int NOT NULL, \
weakest_cnt int NOT NULL,\
strongest_cnt int NOT NULL, \
PRIMARY KEY(idx) \
)"

execute_sql(sql)

()

In [10]:
# 테이블에 한글 문자 입력 가능하도록 변경
sql = "ALTER TABLE bws_test convert to charset utf8;"  

execute_sql(sql)

()

In [16]:
sql = "CREATE TABLE bws_table(idx int NOT NULL, \
eng_text varchar(1000) NOT NULL, \
kor_text varchar(1000) NOT NULL, \
cnt int NOT NULL, \
weakest_cnt int NOT NULL,\
strongest_cnt int NOT NULL, \
PRIMARY KEY(idx) \
)"

execute_sql(sql)

()

In [17]:
# 테이블에 한글 문자 입력 가능하도록 변경
sql = "ALTER TABLE bws_table convert to charset utf8;"  

execute_sql(sql)

()

In [18]:
sql = 'DESC bws_table;'

execute_sql(sql)

(('idx', 'int', 'NO', 'PRI', None, ''),
 ('eng_text', 'varchar(1000)', 'NO', '', None, ''),
 ('kor_text', 'varchar(1000)', 'NO', '', None, ''),
 ('cnt', 'int', 'NO', '', None, ''),
 ('weakest_cnt', 'int', 'NO', '', None, ''),
 ('strongest_cnt', 'int', 'NO', '', None, ''))

In [11]:
file = open(os.path.join(data_path, 'bws_db.csv'),'r', encoding='UTF8')
fReader = csv.reader(file)

In [12]:
idx_list = []

for idx, line in enumerate(fReader):
    if idx == 0: 
        continue 
    
    try:    # ' -> 제거하지 않으면 오류 발생 
        sql = "INSERT INTO bws_test VALUES('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"\
        .format(line[0], line[1].replace("'", ''), line[2].replace("'",''), line[3], line[4], line[5])
        curs.execute(sql)
    except:
        idx_list.append(idx)
        continue     

len(idx_list)

0

In [20]:
idx_list = []

for idx, line in enumerate(fReader):
    if idx == 0: 
        continue 
    
    try:    # ' -> 제거하지 않으면 오류 발생 
        sql = "INSERT INTO bws_table VALUES('{0}', '{1}', '{2}', '{3}', '{4}', '{5}')"\
        .format(line[0], line[1].replace("'", ''), line[2].replace("'",''), line[3], line[4], line[5])
        curs.execute(sql)
    except:
        idx_list.append(idx)
        continue     

len(idx_list)

0

In [21]:
idx

1600

In [13]:
sql = 'SELECT * FROM bws_test;'
execute_sql(sql)

((0,
  'male aged  i used to feel depressed since the age of  in a classic sense',
  '남자 나이 나는 비정상적인 의미에서 나이 이후로 우울함을 느끼곤 했다',
  0,
  0,
  0),
 (1,
  'i am not really sure why i am depressed i am not popular by any means but i have friends',
  '나는 내가 왜 우울한지 잘 모르겠다. 나는 결코 인기가 없지만 나는 친구들이 있다',
  0,
  0,
  0),
 (2,
  'i dont know if i can really give you any solid advice as i am depressed myself',
  '나 자신이 우울하기 때문에 내가 정말로 너에게 확실한 조언을 해줄 수 있을지 모르겠다',
  0,
  0,
  0),
 (3,
  'which is the same when i am depressed but at least before the med adjustment i could genuinely feel happy for a little while',
  '그것은 내가 우울할 때도 마찬가지이지만 적어도 의학적 적응 전에 나는 잠시 동안 진정으로 행복을 느낄 수 있었다',
  0,
  0,
  0),
 (4,
  'i dont think i am depressed i just suck',
  '나는 내가 우울하다고 생각하지 않아. 그냥 형편없어',
  0,
  0,
  0),
 (5,
  'you may want to phrase it like i feel like i am depressed',
  '당신은 내가 우울해 하는 것처럼 표현하고 싶을지도 모른다',
  0,
  0,
  0),
 (6,
  'also i do not want to tell my family i am depressed',
  '또한 나는 가족들에게 내가 우울하다고 말하고 싶지 

In [14]:
curs.fetchall()
curs.close()
conn.commit()
conn.close()

In [24]:
conn = pymysql.connect(host='localhost', user='cori', password='Dhwoehd12!', db='bws', charset='utf8')
curs = conn.cursor()

In [25]:
import os
import json

In [4]:
config_path = './Flask/config'

In [5]:
db_config = {'host': 'localhost', 'user': 'cori', 'password': 'Dhwoehd12!', 'db': 'bws'}

In [6]:
db_config

{'host': 'localhost', 'user': 'cori', 'password': 'Dhwoehd12!', 'db': 'bws'}

In [7]:
with open(os.path.join(config_path, 'db_config.json'), 'w') as f:
    json.dump(db_config, f)

In [27]:
bws_set[7]

{'Q1': [532, 271, 497, 1217],
 'Q2': [607, 835, 194, 1218],
 'Q3': [654, 1165, 455, 1592],
 'Q4': [628, 660, 597, 1395],
 'Q5': [88, 727, 1034, 1263],
 'Q6': [275, 634, 469, 1471],
 'Q7': [327, 335, 12, 1529],
 'Q8': [352, 1030, 653, 1446],
 'Q9': [168, 171, 968, 1556],
 'Q10': [645, 937, 774, 1449],
 'Q11': [584, 588, 575, 1364],
 'Q12': [619, 590, 11, 1400],
 'Q13': [470, 190, 351, 1242],
 'Q14': [1089, 52, 85, 1359],
 'Q15': [256, 762, 934, 1459],
 'Q16': [716, 860, 620, 1339],
 'Q17': [1074, 375, 823, 1581],
 'Q18': [1129, 436, 227, 1410],
 'Q19': [1197, 676, 86, 1352],
 'Q20': [714, 528, 713, 1322],
 'Q21': [489, 1028, 1185, 1330],
 'Q22': [647, 673, 369, 1498],
 'Q23': [440, 555, 971, 1337],
 'Q24': [760, 109, 594, 1565],
 'Q25': [411, 1069, 79, 1578],
 'Q26': [954, 539, 1130, 1411],
 'Q27': [257, 1086, 598, 1259],
 'Q28': [885, 270, 435, 1286],
 'Q29': [582, 250, 1023, 1494],
 'Q30': [248, 208, 290, 1426],
 'Q31': [454, 1140, 626, 1487],
 'Q32': [66, 976, 108, 1297],
 'Q33': [85

In [28]:
bws_set_df = pd.DataFrame(bws_set[0].keys(), columns=['Question_no'])
bws_set_df['idx'] = bws_set_df.index
bws_set_df = bws_set_df[['idx', 'Question_no']]
bws_set_df['Set_no'] = 1
bws_set_df

,idx,Question_no,Set_no
0,0,Q1,1
1,1,Q2,1
2,2,Q3,1
3,3,Q4,1
4,4,Q5,1
...,...,...,...
395,395,Q396,1
396,396,Q397,1
397,397,Q398,1
398,398,Q399,1


In [29]:
bws_set_df['idx1'] = list(map(lambda x: x[0], list(bws_set[0].values())))
bws_set_df['idx2'] = list(map(lambda x: x[1], list(bws_set[0].values())))
bws_set_df['idx3'] = list(map(lambda x: x[2], list(bws_set[0].values())))
bws_set_df['idx4'] = list(map(lambda x: x[3], list(bws_set[0].values())))

In [30]:
bws_set_df

,idx,Question_no,Set_no,idx1,idx2,idx3,idx4
0,0,Q1,1,322,716,598,1456
1,1,Q2,1,584,913,715,1559
2,2,Q3,1,293,775,127,1267
3,3,Q4,1,529,528,165,1450
4,4,Q5,1,740,925,1016,1211
...,...,...,...,...,...,...,...
395,395,Q396,1,731,817,1158,1388
396,396,Q397,1,1109,325,84,1538
397,397,Q398,1,1042,1057,492,1536
398,398,Q399,1,970,58,495,1487


In [31]:
s_idx = 400

for idx in range(1, 8):
    set_idx = list(range(s_idx, s_idx + 400))
    set_df = pd.DataFrame(bws_set[idx].keys(), columns=['Question_no'])
    set_df['idx'] = set_idx
    set_df['Set_no'] = idx + 1 
    set_df['idx1'] = list(map(lambda x: x[0], list(bws_set[idx].values())))
    set_df['idx2'] = list(map(lambda x: x[1], list(bws_set[idx].values())))
    set_df['idx3'] = list(map(lambda x: x[2], list(bws_set[idx].values())))
    set_df['idx4'] = list(map(lambda x: x[3], list(bws_set[idx].values())))
    set_df = set_df[['idx', 'Question_no', 'Set_no', 'idx1', 'idx2', 'idx3', 'idx4']]
    bws_set_df = pd.concat([bws_set_df, set_df])
    s_idx += 400 

In [32]:
bws_set_df

,idx,Question_no,Set_no,idx1,idx2,idx3,idx4
0,0,Q1,1,322,716,598,1456
1,1,Q2,1,584,913,715,1559
2,2,Q3,1,293,775,127,1267
3,3,Q4,1,529,528,165,1450
4,4,Q5,1,740,925,1016,1211
...,...,...,...,...,...,...,...
395,3195,Q396,8,651,1057,1084,1266
396,3196,Q397,8,614,1068,540,1282
397,3197,Q398,8,261,1010,119,1203
398,3198,Q399,8,687,699,145,1314


In [33]:
bws_set_df.to_csv(os.path.join(data_path, 'bws_set.csv'), index=False)

In [34]:
sql = "DROP TABLE bws_set;"
execute_sql(sql)

()

In [35]:
sql = "CREATE TABLE bws_set(idx int NOT NULL, \
Question_no varchar(100) NOT NULL, \
Set_no int NOT NULL, \
idx1 int NOT NULL, \
idx2 int NOT NULL,\
idx3 int NOT NULL, \
idx4 int NOT NULL, \
PRIMARY KEY(idx) \
)"

execute_sql(sql)

()

In [36]:
file = open(os.path.join(data_path, 'bws_set.csv'),'r', encoding='UTF8')
fReader = csv.reader(file)

In [37]:
for idx, line in enumerate(fReader):
    if idx == 0: 
        continue 
    
    try:  
        sql = "INSERT INTO bws_set VALUES('{0}', '{1}', '{2}', '{3}', '{4}', '{5}', '{6}')"\
        .format(line[0], line[1], line[2], line[3], line[4], line[5], line[6])
        curs.execute(sql)
    except:
        continue     

In [38]:
sql = "SELECT * FROM bws_set;"
execute_sql(sql)

((0, 'Q1', 1, 322, 716, 598, 1456),
 (1, 'Q2', 1, 584, 913, 715, 1559),
 (2, 'Q3', 1, 293, 775, 127, 1267),
 (3, 'Q4', 1, 529, 528, 165, 1450),
 (4, 'Q5', 1, 740, 925, 1016, 1211),
 (5, 'Q6', 1, 503, 940, 1105, 1413),
 (6, 'Q7', 1, 0, 653, 109, 1521),
 (7, 'Q8', 1, 1035, 668, 634, 1342),
 (8, 'Q9', 1, 11, 607, 1071, 1313),
 (9, 'Q10', 1, 714, 593, 905, 1526),
 (10, 'Q11', 1, 620, 681, 1120, 1471),
 (11, 'Q12', 1, 692, 294, 396, 1373),
 (12, 'Q13', 1, 132, 1129, 820, 1210),
 (13, 'Q14', 1, 910, 672, 381, 1272),
 (14, 'Q15', 1, 113, 937, 98, 1488),
 (15, 'Q16', 1, 1168, 140, 729, 1286),
 (16, 'Q17', 1, 721, 941, 734, 1530),
 (17, 'Q18', 1, 892, 964, 850, 1269),
 (18, 'Q19', 1, 1089, 539, 469, 1235),
 (19, 'Q20', 1, 432, 63, 407, 1411),
 (20, 'Q21', 1, 532, 709, 522, 1401),
 (21, 'Q22', 1, 1075, 97, 673, 1335),
 (22, 'Q23', 1, 626, 967, 348, 1277),
 (23, 'Q24', 1, 525, 1156, 974, 1558),
 (24, 'Q25', 1, 719, 296, 662, 1207),
 (25, 'Q26', 1, 700, 534, 346, 1421),
 (26, 'Q27', 1, 94, 255, 11

In [39]:
curs.fetchall()
curs.close()
conn.commit()
conn.close()